# HOMEWORK 2
Qdrant uses fastembed under the hood to turn text into vectors.

We will now explore this library

Make sure it's installed:

`pip install fastembed`

## Q1. Embedding the query
Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

- -0.51
- -0.11
- 0
- 0.51

In [1]:
from fastembed import TextEmbedding

/Users/anicol11/ASU Dropbox/Andrea Nicolas Flores/Mac/Documents/projects/LLM-zoomcamp/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set up embedding model
model_name = 'jinaai/jina-embeddings-v2-small-en'
model  = TextEmbedding(model_name=model_name)


In [3]:
# embed a query and find out the length
query = 'I just discovered the course. Can I join now?'


# `embed` returns a generator, so convert it to a list to access the embedding
embedding = list(model.embed([query]))[0] # this is a np array

In [4]:
print(f"min value is: {embedding.min():.2f}")

min value is: -0.12


## A1
minimum value on the embedding is -0.12, so we grab -0.11

## Cosine similarity
The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the norm function:

```bash
import numpy as np
np.linalg.norm(q)
```
Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```bash
q.dot(q)
```

## Q2. Cosine similarity with another vector
Now let's embed this document:

`doc = 'Can I still join the course after the start date?'`

What's the cosine similarity between the vector for the query and the vector for the document?

- 0.3
- 0.5
- 0.7
- 0.9

In [5]:
import numpy as np

In [6]:
# embed a query and find out the length
query_2 = 'Can I still join the course after the start date?'
embedding_2 = list(model.embed([query_2]))[0] # this is a np array


In [7]:
# calculate dot product 
cosine_similarity = embedding.dot(embedding_2)
print(f"cosine similarity: {cosine_similarity:.2f}")

cosine similarity: 0.90


## A2. 

Cosine similarity is 0.90

## Q3. Ranking by cosine
For Q3 and Q4 we will use these documents:


In [8]:

documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:

In [9]:
# get the text only and convert to a vector each
embedding_docs = []
for document in documents:
    doc = document['text']
    embedding_doc = list(model.embed([doc]))[0] # this is a np array
    embedding_docs.append(embedding_doc)

# convert list to a matrix
embedding_docs = np.stack(embedding_docs)

print(embedding_docs.shape)


(5, 512)


In [10]:
# do a dot product between this matrix and the original query one
cosine_similarities = embedding_docs.dot(embedding)
best_index = np.argmax(cosine_similarities)

print(f"Highest similarity: {cosine_similarities[best_index]:.2f}")
print(f"Index of highest similarity: {best_index}")
print(cosine_similarities)


Highest similarity: 0.82
Index of highest similarity: 1
[0.76296845 0.81823783 0.80853973 0.71330786 0.73044994]


## A3.
The index of highest similarity was `1`, with a cosine similarity of `0.82`.

## Q4. Ranking by cosine, version two
Now let's calculate a new field, which is a concatenation of question and text:

```bash
full_text = doc['question'] + ' ' + doc['text']
```


Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

- 0
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

In [11]:
# get the questions + text and convert to a vector each
embedding_docs = []
for document in documents:
    query_doc = document['question'] + ' ' + document['text']
    embedding_doc = list(model.embed([query_doc]))[0] # this is a np array
    embedding_docs.append(embedding_doc)

# convert list to a matrix
embedding_docs = np.stack(embedding_docs)

print(embedding_docs.shape)

(5, 512)


In [12]:
# do a dot product between this matrix and the original query one
cosine_similarities = embedding_docs.dot(embedding)
best_index = np.argmax(cosine_similarities)

print(f"Highest similarity: {cosine_similarities[best_index]:.2f}")
print(f"Index of highest similarity: {best_index}")
print(cosine_similarities)

Highest similarity: 0.85
Index of highest similarity: 0
[0.85145431 0.84365941 0.84082872 0.77551577 0.8086008 ]


## A4.
The index of highest similarity was `0`, with a cosine similarity of `0.85`. It makes sense since that question is pretty much a variant of the question `I just discovered the course. Can I join now?`. Adding the text does not really change things. IT is different from Q3 because on that one, we embedded only the description, but the title was actaully the one key bit of info needed.

## Q5. Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

- 128
- 256
- 384
- 512

## A5.

According to their documentation [here](https://qdrant.github.io/fastembed/examples/Supported_Models/#supported-text-embedding-models) their smallest dimensionality is `384`.


One of these models is `BAAI/bge-small-en`. Let's use it.

## Q6. Indexing with qdrant (2 points)
For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

In [13]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


Add them to qdrant using the model from Q5.

When adding the data, use both question and answer fields:

```bash
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67

In [14]:
# we are cheating a little bit and we are just using the specific model, straight up here
# we shall use quadrant later

# set up embedding model
model_name = 'BAAI/bge-small-en'
model  = TextEmbedding(model_name=model_name)

In [15]:
# embed a query and find out the length
query = 'I just discovered the course. Can I join now?'


# `embed` returns a generator, so convert it to a list to access the embedding
embedding = list(model.embed([query]))[0] # this is a np array

In [16]:
# get the questions + text and convert to a vector each
embedding_docs = []
for document in documents:
    query_doc = document['question'] + ' ' + document['text']
    embedding_doc = list(model.embed([query_doc]))[0] # this is a np array
    embedding_docs.append(embedding_doc)

# convert list to a matrix
embedding_docs = np.stack(embedding_docs)

print(embedding_docs.shape)

# do a dot product between this matrix and the original query one
cosine_similarities = embedding_docs.dot(embedding)
best_index = np.argmax(cosine_similarities)

print(f"Highest similarity: {cosine_similarities[best_index]:.2f}")
print(f"Index of highest similarity: {best_index}")
print(cosine_similarities)

(375, 384)
Highest similarity: 0.87
Index of highest similarity: 14
[0.8147728  0.8197734  0.80301535 0.7736218  0.8234947  0.8414749
 0.8691884  0.79737043 0.80610526 0.8455939  0.78400755 0.83213025
 0.7650218  0.8683312  0.8703173  0.82244146 0.7965847  0.857154
 0.834791   0.7781333  0.8210681  0.79621434 0.8034544  0.77217543
 0.7377288  0.72783285 0.7611208  0.7685133  0.68989664 0.771853
 0.7180948  0.756606   0.76177716 0.749992   0.7433283  0.77547944
 0.70923704 0.69377816 0.7284203  0.7256007  0.6805362  0.71082664
 0.7001067  0.7400493  0.7889377  0.71736807 0.7712903  0.7459129
 0.75641924 0.72377646 0.8195713  0.7361079  0.74429923 0.7656276
 0.7213968  0.7175728  0.73007864 0.73602295 0.7265439  0.7624037
 0.754902   0.8058157  0.81823945 0.7417751  0.72125655 0.7388073
 0.7413675  0.7293712  0.7775129  0.7324535  0.7208928  0.7263732
 0.69528395 0.6863812  0.6722955  0.7165996  0.76008093 0.70321953
 0.68104416 0.6858498  0.7549139  0.7041918  0.7169024  0.7796075
 0.78

In [17]:
documents[best_index]

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp'}

## A6.

We used the smaller embedding model and just did the vector match here on python.

We found that the best match has a cosine similarity of `0.87` and it is for index 14 which relates to the question `The course has already started. Can I still join it?`.